In [45]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk

In [46]:
from sklearn.model_selection import train_test_split

dataset_path = './dataset'
target_column = 'target_reg'

train_df = pd.read_csv(f'{dataset_path}/train.csv')
train_df.dropna(inplace=True)
print(train_df.isna().sum().sum())

0


In [47]:
# from sklearn.preprocessing import OneHotEncoder
# megye_2014_df = train_df["MEGYE_2014"]
# encoder = OneHotEncoder(handle_unknown='ignore')
# encoder_df = pd.DataFrame(encoder.fit_transform(train_df[["MEGYE_2014"]]).toarray())
# print(encoder_df)
# print(encoder_df.isna().sum().sum())

In [48]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')

columns_to_onehotencode = [
    'MEGYE_2014',
    # 'MEGYE_2015',
    # 'MEGYE_2016',
    # 'SZEKHELY_IR_SZAM_2014',
    # 'SZEKHELY_IR_SZAM_2015',
    # 'SZEKHELY_IR_SZAM_2016',
    # 'GAZDALKODASI_FORMA_2014_fo_kategoria',
	# 'GAZDALKODASI_FORMA_2014_1_alkategoria',
    # 'GAZDALKODASI_FORMA_2014_2_alkategoria',
    # 'GAZDALKODASI_FORMA_2015_fo_kategoria',
    # 'GAZDALKODASI_FORMA_2015_1_alkategoria',
    # 'GAZDALKODASI_FORMA_2015_2_alkategoria',
    # 'GAZDALKODASI_FORMA_2016_fo_kategoria',
    # 'GAZDALKODASI_FORMA_2016_1_alkategoria',
    # 'GAZDALKODASI_FORMA_2016_2_alkategoria',
    # 'KKV_BESOROLAS_2014_fo_kategoria',
    # 'KKV_BESOROLAS_2014_1_alkategoria',
    # 'KKV_BESOROLAS_2014_2_alkategoria',
    # 'KKV_BESOROLAS_2015_fo_kategoria',
    # 'KKV_BESOROLAS_2015_1_alkategoria',
    # 'KKV_BESOROLAS_2015_2_alkategoria',
    # 'KKV_BESOROLAS_2016_fo_kategoria',
    # 'KKV_BESOROLAS_2016_1_alkategoria',
    # 'KKV_BESOROLAS_2016_2_alkategoria',
    # 'agazat_2014_fo_kategoria',
    # 'agazat_2014_1_alkategoria',
    # 'agazat_2014_2_alkategoria',
    # 'agazat_2014_3_alkategoria',
    # 'agazat_2015_fo_kategoria',
    # 'agazat_2015_1_alkategoria',
    # 'agazat_2015_2_alkategoria',
    # 'agazat_2015_3_alkategoria',
    # 'agazat_2016_fo_kategoria',
    # 'agazat_2016_1_alkategoria',
    # 'agazat_2016_2_alkategoria',
    # 'agazat_2016_3_alkategoria'
]

encoder_df = pd.DataFrame(encoder.fit_transform(train_df[[*columns_to_onehotencode]]).toarray())

train_df = pd.concat([train_df, encoder_df], axis='columns')
print(train_df.shape)
print(train_df.isna().sum().sum())

(33079, 152)
202768


In [49]:
is_NaN = train_df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = train_df[row_has_NaN]

print(rows_with_NaN)

            id  target_reg  KATEGORIKUS_VALTOZO_2014  \
31745  31746.0         0.0                       0.0   
31746  31747.0         0.0                       1.0   
31747  31748.0         0.0                       0.0   
31748  31749.0         0.0                       1.0   
31749  31750.0         0.0                       0.0   
...        ...         ...                       ...   
31687      NaN         NaN                       NaN   
31691      NaN         NaN                       NaN   
31727      NaN         NaN                       NaN   
31728      NaN         NaN                       NaN   
31736      NaN         NaN                       NaN   

       KATEGORIKUS_VALTOZO_2015  KATEGORIKUS_VALTOZO_2016 MEGYE_2014  \
31745                       0.0                       0.0     Somogy   
31746                       1.0                       0.0   Budapest   
31747                       0.0                       0.0   Budapest   
31748                       0.0        

In [50]:
target_df = train_df[target_column]
train_df.drop([target_column, *columns_to_onehotencode], axis='columns', inplace=True)

train_df.columns = train_df.columns.map(str)

x_train, x_test, y_train, y_test = train_test_split(train_df, target_df, random_state=42)

In [51]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)

predictions = lin_reg.predict(x_test)

mse = mean_squared_error(predictions, y_test)
print(f'mean squared error: {mse}')

ValueError: could not convert string to float: 'Budapest'